# Criando um exemplo real: Syngenta - sementes NK

## Importando as bibliotecas necessárias:


In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests

# Definindo funções:

In [224]:
def abre_e_scrap(url):
    
    """Esta função abre o url dado como parâmetro e retorna a 'sopa de html'.
    Parâmetro: url"""
    
    html = urlopen(url)
    scrap = BeautifulSoup(html, 'html.parser')
    return scrap.prettify()

In [228]:
#teste da função abre_e_scrap()

url = 'http://www.portalsyngenta.com.br/sementes/nk-soja'
scrap = abre_e_scrap(url)

### Explorado a página html com 6 cards de mesma estrutura: < div class = card-text-2
#### Criando uma função que busca com `find_all` e retorna uma lista com os links de cada card

In [229]:
# Buscando no html os cards repetidos, com <div class: "class_card"

def find_links_in_cards(scrap, class_card):
    
    """ Função que recebe a sopa e a string dos cards, 
    passa por todos esses cards e retorna os links contidos neles.
    Parâmetros: 
    scrap: página html obtida através do BeautifulSoup
    class_card: qual a str correspondente ao card a ser encontrado
    Retorna: uma lista contendo os links contidos em cada card."""
    
    cards = scrap.find_all('div', {'class': class_card})
    
    links = []

    for link in cards:
        link = link.a['href']
        links.append(link)
    
    return links

In [234]:
# testando a função find_links_in_cards

class_card = "card-text-2 card-portfolio"
links = find_links_in_cards(scrap, class_card)
links[:2]

['/sementes/nk-soja/nk-8770-ipro', '/sementes/nk-soja/nk-7777-ipro']

## Dependendo do link obtido, vai ser necessário complementá-lo:

In [235]:
def complement_link(lista_links, link_principal):
    
    """Função que recebe uma lista com links parciais e complementa-os usando o link principal.
    Parâmetros:
    lista_links: lista contendo links parciais
    link_principal: início do link que foi oculto nos links parciais"""
    
    links_completos = []
    
    for link in lista_links:
        link = link_principal+link
        links_completos.append(link)
        
    return links_completos

In [ ]:
# testando a função complement_link:

principal = 'http://www.portalsyngenta.com.br'
links_completos = complement_link(links, principal)

## 3. Varremos os cards com um `for`, para obtermos os links de cada semente NK:
####  < a [ ' href ' ]  ----> armazenando em uma lista
## 4. Juntando o site principal com os links armazenados na lista
#### Os links fornecidos são complementares ao do site principal

## 5. Explorando cada link e coletando as informações:

* inserindo variáveis para facilitar o aproveitamento do código:
  * conteúdos dentro de <div <class
  * `class_titulo`
  * `class_descricao`
               
                                    
* guardando as informações em listas                                   

In [ ]:
class_titulo = "prop-nk-titulo"
class_descricao = "prop-nk-descricao"

class_beneficios = "item"
class_benef_subtitulo = "subtitulo-beneficios"    

tamanho_nome = -12    # no final do link, as últimas 12 letras correspondem ao nome

lista_titulos = []
lista_descricao = []
lista_nome = []
lista_beneficios = []
lista_benef_subtitulo = []


for link in links:
    url = requests.get(link)
    conteudo = url.content
    scrap = BeautifulSoup(conteudo, 'html.parser')   # soup de cada página
    
    lista_nome.append(link[tamanho_nome:])          # nome das sementes
    
    titulo = scrap.find_all('div', {'class': class_titulo})
    descricao = scrap.find_all('div', {'class': class_descricao})
    beneficios = scrap.find_all('div', {'class': class_beneficios})
    benef_subtitulo = scrap.find_all('p', {'class': class_benef_subtitulo})
    
    for tit in titulo:
        lista_titulos.append(tit.p.text)
        
    for desc in descricao:
        lista_descricao.append(desc.p.text)    
        
    for benef in beneficios:
        lista_beneficios.append(link[tamanho_nome:])   # nome das sementes
        lista_beneficios.append(benef.p.text)
    
    for sub in benef_subtitulo:
        lista_benef_subtitulo.append(sub.text)

## 6. Criando um array com 6 listas de 9 elementos cada; e inserindo o nome da semente

In [223]:
array = np.array(lista_descricao).reshape(6, 9)

df_caract = pd.DataFrame(data=array)
df_caract['Nome'] = lista_nome
df_caract.head(2)

,0,1,2,3,4,5,6,7,8,Nome
0,semi-determinado,ROXA,8.7,CINZA,Tolerante,Marrom Claro,97 - 112,140 - 190 KG,14 - 19,nk-8770-ipro
1,INDETERMINADO,ROXA,7.7,CINZA,Tolerante,Marrom Claro,80 - 95,190 G,15 - 20,nk-7777-ipro


## 7. Criando DataFrame das caracteristicas + nomes das colunas
* a lista_titulos se repente de 9 em 9 elementos
* pegando os elementos únicos e adicionando 'Nome'
* colocando como header a lista contendo 10 elementos com os nomes das colunas

In [222]:
lista_titulos = lista_titulos[:9].copy()
lista_titulos.append('Nome')
df_caract.columns = lista_titulos
df_caract.head(2)

,Hábito de Crescimento,Cor da Flor,Grupo de Maturação,Pubescência,Acamamento,Hilo,Altura da Planta (cm),"PMS Médio 5,5 mm",Inserção de Vagem (cm),Nome
0,semi-determinado,ROXA,8.7,CINZA,Tolerante,Marrom Claro,97 - 112,140 - 190 KG,14 - 19,nk-8770-ipro
1,INDETERMINADO,ROXA,7.7,CINZA,Tolerante,Marrom Claro,80 - 95,190 G,15 - 20,nk-7777-ipro


## mesmo processo de criação de DataFrame para:
* ### Benefícios
* ### Recomendações

In [83]:
array_benef = np.array(lista_beneficios)
array_benef = array_benef.reshape(21,2)
df_benef = pd.DataFrame(array_benef)

df_benef.columns = ['Nome', 'Beneficios']
df_benef['Subtitulo'] = lista_benef_subtitulo[:21]

## 8. Lendo as tabelas dos sites (html)
* ### Recomendação Técnica

In [ ]:
url = 'http://www.portalsyngenta.com.br/sementes/nk-soja/nk-6201-ipro'
class_table = 'table-sementes'   # o que será procurado na página

table_recom = pd.read_html(url)

html = urlopen(url)
scrap = BeautifulSoup(html, 'html.parser')

table_recom_html = scrap.find('table', {'class': class_table})

lista_temp = []         # lista temporária, antes de tratar as cores
lista_cel_recom = []      # lista pós tratamento, recebe apenas os valores da legenda

for row in table_recom_html:
    for i in row:
        i = str(i)
        if "td-data-gray" in i:
            i = i.replace('"td-data-gray"', '"Não recomendável"')
            i = i.replace('"td-data-gray border-right"', '"Não recomendável"')
            lista_temp.append(i)
        elif 'td-data-light-blue' in i:
            i = i.replace('"td-data-light-blue"', '"Aceitável"')
            i = i.replace('"td-data-light-blue border-right"', '"Aceitável"')
            lista_temp.append(i)
        elif 'td-data-dark-blue' in i:
            i = i.replace('"td-data-dark-blue"', '"Recomendado"')
            i = i.replace('"td-data-dark-blue border-right"', '"Recomendado"')
            lista_temp.append(i)
            
for i in lista_temp:
    i = i.replace('<td class=', '')
    i = i.replace('></td>', '')
    lista_cel_recom.append(i)

In [ ]:
df_recom = pd.DataFrame(table_recom[0])
df_recom

In [ ]:
array_legendas_recom = np.array(lista_cel_recom).reshape(7, 12)


In [ ]:
df_legendas = pd.DataFrame(array_legendas_recom)
df_legendas.columns = ['Setembro', 'Setembro.1', 'Setembro.2',
       'Outubro', 'Outubro.1', 'Outubro.2', 'Novembro', 'Novembro.1',
       'Novembro.2', 'Dezembro', 'Dezembro.1', 'Dezembro.2']

df_legendas

In [ ]:
df_recom.update(df_legendas)
df_recom

## 9. Exportando para arquivo excel, dentro da pasta: planilhas/

In [82]:
df_caract.to_excel('planilhas/Syngenta_NK/caracteristicas.xlsx')
df_benef.to_excel('planilhas/Syngenta_NK/beneficios.xlsx')